Use case: 2 linked windows with "a hundred" candidates to load for each using drop-down menus.

In [ ]:
import os

from ipywidgets import Dropdown, Label, Button, HBox, VBox, Layout

from ginga.misc import Datasrc
from ginga.misc.log import get_logger
from ginga.util import wcsmod
from ginga.util.iohelper import get_fileinfo

from astrowidgets import ImageWidget

In [ ]:
# Reading ASDF in Ginga needs this to be specified early on
wcsmod.use('astropy_ape14')

In [ ]:
logger = get_logger('my viewer', log_stderr=True, log_file=None, level=30)

In [ ]:
statusmsg = Label('')

In [ ]:
class LinkedImageWidget(ImageWidget):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        num_images = 10  # Max images in cache; should user be able to set this?
        self.datasrc = Datasrc.Datasrc(num_images)  # Cache
        self._other_viewer = None
        
    def load_jwst_asdf(self, filename):
        if filename in self.datasrc:
            image = self.datasrc[filename]
        else:
            import asdf
            from ginga.AstroImage import AstroImage

            image = AstroImage(logger=self.logger)
            image.load_file(filename, data_key='data')

            self.datasrc[filename] = image

        self._viewer.set_image(image)

    def load_fits(self, filename):
        bnch = get_fileinfo(filename)
        
        if filename in self.datasrc:
            image = self.datasrc[filename]
            self._viewer.set_image(image)
        else:
            super().load_fits(bnch.filepath, numhdu=bnch.numhdu)
            self.datasrc[filename] = self._viewer.get_image()
        
    def link_viewer(self, other_viewer):
        self._other_viewer = other_viewer

    def _mouse_click_cb(self, viewer, event, data_x, data_y):
        image = viewer.get_image()
        if image is None:  # Nothing to do
            return
              
        if isinstance(self._other_viewer, ImageWidget):
            other_image = self._other_viewer._viewer.get_image()
            if other_image is not None:
                if image.wcs.wcs is not None and other_image.wcs.wcs is not None:
                    ra, dec = image.pixtoradec(data_x, data_y)
                    other_x, other_y = other_image.radectopix(ra, dec)
                    statusmsg.value = f'Clicked RA={ra},Dec={dec}'
                else:
                    other_x, other_y = data_x, data_y
                    statusmsg.value = f'Clicked X={data_x},Y={data_y}'
            
                self._other_viewer._mouse_click_cb(self._other_viewer._viewer, event, other_x, other_y)
            
        super()._mouse_click_cb(self._viewer, event, data_x, data_y)

In [ ]:
w1 = LinkedImageWidget(logger=logger)
w2 = LinkedImageWidget(logger=logger)

In [ ]:
w1.link_viewer(w2)  # w1 is primary viewer for WCS matching
w1.click_center = True
w2.click_center = True

In [ ]:
fits_files = [
    'N/A',
    'D:\\PLLIM\\STScI\\ssb\\test_data\\jw42424001001_01101_00001_nrca5_assign_wcs.fits',
    'D:\\PLLIM\\STScI\\ssb\\test_data\\jw42424001001_01101_00002_nrca5_assign_wcs.fits',
    'D:\\PLLIM\\STScI\\ssb\\test_data\\jw42424001001_01101_00003_nrca5_assign_wcs.fits']

In [ ]:
def menu_factory(label, w):
    def menu(event):
        filename = event['new']
        if filename != 'N/A':
            label.value = os.path.basename(filename)
            # w.load_fits(filename)  # FITS
            w.load_jwst_asdf(filename)  # JWST ASDF-in-FITS
    return menu

In [ ]:
label1 = Label('Filename')
label2 = Label('Filename')
statusmsg.value = ''  # Reset status message

In [ ]:
dropdown1 = Dropdown(
    options=fits_files,
    value='N/A',
    description='Select file:')

dropdown2 = Dropdown(
    options=fits_files,
    value='N/A',
    description='Select file:')

In [ ]:
dropdown1.observe(menu_factory(label1, w1), 'value')
dropdown2.observe(menu_factory(label2, w2), 'value')

In [ ]:
button_zoomlevel = Button(description="Sync zoom level")

def on_button_zoomlevel_clicked(b):
    w2.zoom_level = w1.zoom_level

button_zoomlevel.on_click(on_button_zoomlevel_clicked)

In [ ]:
VBox([HBox([VBox([label1, w1, dropdown1], layout=Layout(margin='0 10px 0 0')),
            VBox([label2, w2, dropdown2])]),
      button_zoomlevel,
      statusmsg])

#### Instructions

Load images by selecting them from the respective dropdown menus for the viewers.

Click on the left viewer and the right one will match in RA and Dec of centered point. No distortion correction is performed at this point.

Press `+`/`-` to zoom in/out. Click "Sync zoom level" button to have the right viewer match the zoom level of the left one.